# Deep Learning with Python

2.3 The gears of neural networks: tensor operations

Aqui vamos observar algumas operações com tensores, como operações elementares, broadcasting, dot, reshaping (muuuuito utilizado!!) e um pouco de geometria. 
Oh! Sem desistir, bora bora ! 💪🏾💪🏾

## Element-wise (operações elementares)

A operação elemento-a-elemento é atuar em cada elemento do tensor separadamente. Por exemplo, implementamos um relu simples (relu (x) = max (x, 0)):

In [3]:
def naive_relu(x):
    assert len(x.shape) == 2    # x é um tensor 2D Numpy.
    x = x.copy()    # Evite sobrescrever o tensor de entrada.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = max(x[i, j], 0)
    return x

A adição também é uma operação baseada no elemento:

In [4]:
def naive_add(x, y):
    # assert x and y are 2D Numpy tensors and have the same shape.
    assert len(x.shape) == 2
    assert x.shape == y.shape
    
    x = x.copy()    # Avoid overwriting the input tensor.
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] += y[i, j]
    return x

Em Numpy, tudo isso é escrito. O cálculo específico é realizado pelo BLAS escrito em C ou Fortran, e a velocidade é muito alta.


Você pode verificar se o BLAS está instalado assim:

In [5]:
import numpy as np

np.show_config()

blas_mkl_info:
    libraries = ['mkl_rt']
    library_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\include']
blas_opt_info:
    libraries = ['mkl_rt']
    library_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\include']
lapack_mkl_info:
    libraries = ['mkl_rt']
    library_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\include']
lapack_opt_info:
    libraries = ['mkl_rt']
    library_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\\Library\\lib']
    define_macros = [('SCIPY_MKL_H', None), ('HAVE_CBLAS', None)]
    include_dirs = ['C:/Users/Lenovo/Anaconda3/envs/tf\

Aqui está como usar relu element-wise do numpy, adicione:

In [6]:

a = np.array([[1, 2, 3],
              [-1, 2, -3],
              [3, -1, 4]])
b = np.array([[6, 7, 8], 
              [-2, -3, 1], 
              [1, 0, 4]])

c = a + b    # Element-wise addition
d = np.maximum(c, 0)    # Element-wise relu

print(c)
print(d)

[[ 7  9 11]
 [-3 -1 -2]
 [ 4 -1  8]]
[[ 7  9 11]
 [ 0  0  0]
 [ 4  0  8]]


### Broadcasting (Transmissão)

Ao realizar operações com elementos, se as formas dos dois tensores forem diferentes, o tensor menor será "transmitido" na mesma forma que o tensor maior, quando viável.

Especificamente, você pode transmitir para duas folhas de forma (a, b, ..., n, n + 1, ..., m) e (n, n + 1, ..., m) Quantidades realizam operações elementares.

Tal como:

In [7]:
x = np.random.random((64, 3, 32, 10))    # x is a random tensor with shape (64, 3, 32, 10).
y = np.random.random((32, 10))    # y is a random tensor with shape (32, 10).
z = np.maximum(x, y)    # The output z has shape (64, 3, 32, 10) like x.

A operação de transmissão é a seguinte:

1.Eixo de aumento de tensor pequeno (eixo de transmissão), adicionar ao mesmo que o grande (ndim)


2.Os elementos do tensor pequeno são repetidos no novo eixo e adicionados à mesma forma que o grande (forma)

É claro que na implementação real, não copiamos dessa forma, o que é um desperdício de espaço, implementamos essa "cópia" diretamente no algoritmo. Por exemplo, implementamos um vetor simples e adição de matriz:

In [8]:
def naive_add_matrix_and_vector(m, v):
    assert len(m.shape) == 2    # m is a 2D Numpy tensor.
    assert len(v.shape) == 1    # v is a Numpy vector.
    assert m.shape[1] == v.shape[0]
    
    m = m.copy()
    for i in range(m.shape[0]):
        for j in range(m.shape[1]):
            m[i, j] += v[j]
    return m

naive_add_matrix_and_vector(np.array([[1 ,2, 3], [4, 5, 6], [7, 8, 9]]), 
                            np.array([1, -1, 100]))

array([[  2,   1, 103],
       [  5,   4, 106],
       [  8,   7, 109]])

### Tensor Dot
Produto escalar tensor (ponto)
O produto escalar do tensor, ou produto do tensor, é feito com o ponto (x, y) em numpy.

A operação do produto escalar pode ser vista a partir do seguinte programa simples:

In [9]:
# Produto escalar vetorial
def naive_vector_dot(x, y):
    assert len(x.shape) == 1
    assert len(y.shape) == 1
    assert x.shape[0] == y.shape[0]
    
    z = 0.
    for i in range(x.shape[0]):
        z += x[i] * y[i]
    return z


# Matriz e produto escalar vetorial
def naive_matrix_vector_dot(x, y):
    z = np.zeros(x.shape[0])
    for i in range(x.shape[0]):
        z[i] = naive_vector_dot(x[i, :], y)
    return z


# Produto escalar matricial
def naive_matrix_dot(x, y):
    assert len(x.shape) == 2
    assert len(y.shape) == 2
    assert x.shape[1] == y.shape[0]
    
    z = np.zeros((x.shape[0], y.shape[1]))
    for i in range(x.shape[0]):
        for j in range(y.shape[1]):
            row_x = x[i, :]
            column_y = y[:, j]
            z[i, j] = naive_vector_dot(row_x, column_y)
    return z

In [10]:
a = np.array([[1, 2, 3],
              [-1, 2, -3],
              [3, -1, 4]])
b = np.array([[6, 7, 8], 
              [-2, -3, 1], 
              [1, 0, 4]])
naive_matrix_dot(a, b)

array([[  5.,   1.,  22.],
       [-13., -13., -18.],
       [ 24.,  24.,  39.]])

O mesmo é verdadeiro para produtos de ponto tensor de alta dimensão. Por exemplo, (trata-se de forma):

(a, b, c, d) . (d,) -> (a, b, c)

(a, b, c, d) . (d, e) -> (a, b, c, e)

### Reshaping
Deformação do tensor (remodelagem)


Essa operação, em suma, ainda são esses elementos, mas a forma como estão dispostos mudou.

In [11]:
x = np.array([[0., 1.],
              [2., 3.],
              [4., 5.]])
print(x.shape)

(3, 2)


In [12]:
x.reshape((6, 1))

array([[0.],
       [1.],
       [2.],
       [3.],
       [4.],
       [5.]])

In [13]:
x.reshape((2, 3))

array([[0., 1., 2.],
       [3., 4., 5.]])

"Transposição" (transposição) é uma transformação especial de matriz. Transposição é a troca de linhas e colunas.


O original x [i,:], após a transposição, torna-se x [:, i].

In [14]:
x = np.zeros((300, 20))
y = np.transpose(x)
print(y.shape)

(20, 300)


Ufa!! Passou! Sobrevivemos? Então vamos logo para o próximo capítulo. Vem vem. 